In [ ]:
# Variational Recovery Map for 5 qubits [ONLY VAR PART]

# beta[0] -> RX(red); beta[1] -> RZ(blue); beta[2] -> RZZ(black)
# L = num repetition -> 3 specified in paper

def decode_variational_5(beta, L = 3, wires=5):
    
    rzz_pairs = [
        (0, 1), (1, 2), (2, 3), (3, 4),
        (0, 2), (2, 4),
        (1, 3), 
        (0, 4), 
        (0, 3),
        (1, 4)
    ]

    for i in range(wires):
        qml.RZ(beta[1], wires=i)

    for l in range(L):
        for i in range(wires):
            qml.RX(beta[0], wires=i)
            qml.RZ(beta[1], wires=i)        

        for (i, j) in rzz_pairs:
            qml.IsingZZ(beta[2], wires=[i, j])

    for i in range(wires):
        qml.RX(beta[0], wires=i)
        qml.RZ(beta[1], wires=i)           

        
        
# Variational Recovery Map for 3 qubits [ONLY VAR PART]

# beta[0] -> RX(red); beta[1] -> RZ(blue); beta[2] -> CZ(black)
# L = num repetition -> 2 specified in paper

def decode_variational_3(beta, L = 2, wires=3):
    
    for i in range(wires):
        qml.RZ(beta[1], wires=i)

    for l in range(L):
        for i in range(wires):
            qml.RX(beta[0], wires=i)
            qml.RZ(beta[1], wires=i)        
    
        qml.CZ(beta[2], wires=[0, 1])
        qml.CZ(beta[2], wires=[1, 2])
        qml.CZ(beta[2], wires=[0, 2])
        

    for i in range(wires):
        qml.RX(beta[0], wires=i)
        qml.RZ(beta[1], wires=i)

In [ ]:
## - - WORK IN PROGRESS -- ##

In [ ]:
import pennylane as qml
import numpy as np

# --- DEVICES ---
dev_sample = qml.device("default.mixed", wires=9, shots=1)
dev_analytic = qml.device("default.mixed", wires=9, shots=1000)

# --- ENCODER ---
def encoding_circuit(wires):
    for wire in wires[1:]:
        qml.CNOT(wires=[0, wire])
    for wire in wires:
        qml.Hadamard(wires=wire)
    
    qml.IsingZZ(-np.pi/2, wires=[4, 0])
    qml.IsingZZ(-np.pi/2, wires=[0, 1])
    qml.IsingZZ(-np.pi/2, wires=[1, 2])
    qml.IsingZZ(-np.pi/2, wires=[2, 3])
    qml.IsingZZ(-np.pi/2, wires=[3, 4])


def apply_noise_layer():
#     qml.PauliX(wires=0) 
    qml.AmplitudeDamping(1, wires=0)

# --- SYNDROME MEASUREMENT ---
def measure_syndrome():
    
    # S1 = X Z Z X I
    qml.Hadamard(5)
    qml.Hadamard(0); qml.Hadamard(3)
    for i in [0, 1, 2, 3]:
        qml.CNOT(wires=[i, 5])
    qml.Hadamard(0); qml.Hadamard(3)

    # S2 = I X Z Z X
    qml.Hadamard(6)
    qml.Hadamard(1); qml.Hadamard(4)
    for i in [1, 2, 3, 4]:
        qml.CNOT(wires=[i, 6])
    qml.Hadamard(1); qml.Hadamard(4)

    # S3 = X I X Z Z
    qml.Hadamard(7)
    qml.Hadamard(0); qml.Hadamard(2)
    for i in [0, 2, 3, 4]:
        qml.CNOT(wires=[i, 7])
    qml.Hadamard(0); qml.Hadamard(2)

    # S4 = Z X I X Z
    qml.Hadamard(8)
    qml.Hadamard(1); qml.Hadamard(3)
    for i in [0, 1, 3, 4]:
        qml.CNOT(wires=[i, 8])
    qml.Hadamard(1); qml.Hadamard(3)
    

# --- CORRECTION MAPPING ---
def decode_and_correct(syndrome):
    syndrome_map = {
        (0, 0, 0, 0): None,
        (0, 0, 0, 1): (0, 'X'), (1, 0, 0, 0): (1, 'X'), 
        (1, 1, 0, 0): (2, 'X'), (0, 1, 1, 0): (3, 'X'), 
        (0, 0, 1, 1): (4, 'X'), (1, 0, 1, 0): (0, 'Z'), 
        (0, 1, 0, 1): (1, 'Z'), (0, 0, 1, 0): (2, 'Z'), 
        (1, 0, 0, 1): (3, 'Z'), (0, 1, 1, 1): (4, 'Z'),
        (1, 0, 1, 1): (0, 'Y'), (1, 1, 0, 1): (1, 'Y'), 
        (1, 1, 1, 0): (2, 'Y'), (1, 1, 1, 1): (3, 'Y'), 
        (0, 1, 0, 0): (4, 'Y')
    }
    key = tuple(syndrome)
    if key not in syndrome_map or syndrome_map[key] is None:
        return []
    qubit, op = syndrome_map[key]
    return [getattr(qml, f'Pauli{op}')(wires=qubit)]


@qml.qnode(dev_sample)
def apply_noise_and_measure():
    encoding_circuit(dev_sample.wires)
    apply_noise_layer()
    measure_syndrome()
    return qml.sample(wires=[5, 6, 7, 8])


@qml.qnode(dev_sample)
def apply_correction(correction_ops=[]):
    for op in correction_ops:
        op.queue()
    return qml.density_matrix(wires=[0])  # logical qubit


# --- IDEAL Logical |0>  ---
@qml.qnode(dev_analytic)
def ideal_logical_zero():
    return qml.density_matrix(wires=[0])

# --- MAIN TEST ---
if __name__ == '__main__':
    for i in range(15):
        syndrome = apply_noise_and_measure()
        print("Syndrome:", list(syndrome))

        correction_ops = decode_and_correct(list(syndrome))
        print("Correction ops:", correction_ops)

        recovered_rho = apply_correction(correction_ops)
        ideal_rho = ideal_logical_zero()

        fidelity = qml.math.fidelity(recovered_rho, ideal_rho)
        print("Fidelity after recovery:", fidelity)
        print("-" * 40)


In [ ]:
import pennylane as qml
import numpy as np

# --- DEVICES ---
dev_sample = qml.device("default.mixed", wires=9, shots=1)
dev_analytic = qml.device("default.mixed", wires=9, shots=1000)

# --- ENCODER ---
def encoding_circuit(wires):
    for wire in wires[1:]:
        qml.CNOT(wires=[0, wire])
    for wire in wires:
        qml.Hadamard(wires=wire)
    
    qml.IsingZZ(-np.pi/2, wires=[4, 0])
    qml.IsingZZ(-np.pi/2, wires=[0, 1])
    qml.IsingZZ(-np.pi/2, wires=[1, 2])
    qml.IsingZZ(-np.pi/2, wires=[2, 3])
    qml.IsingZZ(-np.pi/2, wires=[3, 4])


def apply_noise_layer(gamma):
#     qml.PauliX(wires=0) 
    qml.AmplitudeDamping(gamma, wires=0)

# --- SYNDROME MEASUREMENT ---
def measure_syndrome():
    
    # S1 = X Z Z X I
    qml.Hadamard(5)
    qml.Hadamard(0); qml.Hadamard(3)
    for i in [0, 1, 2, 3]:
        qml.CNOT(wires=[i, 5])
    qml.Hadamard(0); qml.Hadamard(3)

    # S2 = I X Z Z X
    qml.Hadamard(6)
    qml.Hadamard(1); qml.Hadamard(4)
    for i in [1, 2, 3, 4]:
        qml.CNOT(wires=[i, 6])
    qml.Hadamard(1); qml.Hadamard(4)

    # S3 = X I X Z Z
    qml.Hadamard(7)
    qml.Hadamard(0); qml.Hadamard(2)
    for i in [0, 2, 3, 4]:
        qml.CNOT(wires=[i, 7])
    qml.Hadamard(0); qml.Hadamard(2)

    # S4 = Z X I X Z
    qml.Hadamard(8)
    qml.Hadamard(1); qml.Hadamard(3)
    for i in [0, 1, 3, 4]:
        qml.CNOT(wires=[i, 8])
    qml.Hadamard(1); qml.Hadamard(3)
    

# --- CORRECTION MAPPING ---
def decode_and_correct(syndrome):
    syndrome_map = {
        (0, 0, 0, 0): None,
        (0, 0, 0, 1): (0, 'X'), (1, 0, 0, 0): (1, 'X'), 
        (1, 1, 0, 0): (2, 'X'), (0, 1, 1, 0): (3, 'X'), 
        (0, 0, 1, 1): (4, 'X'), (1, 0, 1, 0): (0, 'Z'), 
        (0, 1, 0, 1): (1, 'Z'), (0, 0, 1, 0): (2, 'Z'), 
        (1, 0, 0, 1): (3, 'Z'), (0, 1, 1, 1): (4, 'Z'),
        (1, 0, 1, 1): (0, 'Y'), (1, 1, 0, 1): (1, 'Y'), 
        (1, 1, 1, 0): (2, 'Y'), (1, 1, 1, 1): (3, 'Y'), 
        (0, 1, 0, 0): (4, 'Y')
    }
    key = tuple(syndrome)
    if key not in syndrome_map or syndrome_map[key] is None:
        return []
    qubit, op = syndrome_map[key]
    return [getattr(qml, f'Pauli{op}')(wires=qubit)]


@qml.qnode(dev_sample)
def apply_noise_and_measure(gamma):
    encoding_circuit(dev_sample.wires)
    apply_noise_layer(gamma)
    measure_syndrome()
    return qml.sample(wires=[5, 6, 7, 8])


@qml.qnode(dev_sample)
def apply_correction(correction_ops=[]):
    for op in correction_ops:
        op.queue()
    return qml.density_matrix(wires=[0])  # logical qubit


# --- IDEAL Logical |0>  ---
@qml.qnode(dev_analytic)
def ideal_logical_zero():
    return qml.density_matrix(wires=[0])

# --- MAIN TEST ---
if __name__ == '__main__':
    gamma = 0
    while(gamma<=0.5):
        fid = 0
        
        for i in range(50):
            syndrome = apply_noise_and_measure(gamma)
    #             print("Syndrome:", list(syndrome))

            correction_ops = decode_and_correct(list(syndrome))
    #             print("Correction ops:", correction_ops)

            recovered_rho = apply_correction(correction_ops)
            ideal_rho = ideal_logical_zero()

            fidelity = qml.math.fidelity(recovered_rho, ideal_rho)
#             print("Fidelity after recovery:", fidelity)
    #             print("-" * 40)
            fid += fidelity

        print("gamma = ", gamma, "; avg fid = ", (fid/50), "\n")
        gamma+=0.1